# MVP

* reference panel: `download_1000G()` in `bigsnpr`

Including 2490 (mostly unrelated) individuals and ~1.7M SNPs in common with either HapMap3 or the UK Biobank.

* summary stats: raw and posterior betas for traits ldl,hdl,tc and tg

* LDpred2-auto model free of hyper-parameters

The model I used to estimate effect size is auto mdoel. It runs the algorithm for 30 different p (the proportion of causal variants) values range from 10e-4 to 0.9, and heritability from LD score regression as initial value.



* Important to check convergence

See the convergence path of p (proportion of causal variants) and $h^2$ heritability.

1336793 over 1339339 were removed in Quality Control.

## Step 1: common snps

In [12]:
sos run ldpred.ipynb common_snp \
    --stat_snp mvpdata/pos_sumstats_hdl.snplist \
    --ref_snp 1000G/1000G.QC.snplist \
    --test_snp ukbiobank/UKB.TMP.snplist \
    --summstats_file mvpdata/pos_sumstats_tg.rds \
    --sub_stats mvpdata/pos_sumstats_hdl.SUB.rds \
    --stats_comsnp mvpdata/common.snplist \
    --test_comsnp ukbiobank/common.snplist \
    --ref_comsnp 1000G/common.snplist

INFO: Running common_snp: 2004l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
INFO: common_snp is completed.
INFO: common_snp output:   mvpdata/pos_sumstats_hdl.SUB.rds mvpdata/common.snplist... (4 items)
INFO: Workflow common_snp (ID=we3bb041313e5ee79) is executed successfully with 1 completed step.


: 1

## Step 2: subsetting reference panel

In [22]:
sos run ldpred.ipynb subsets \
    --bed_file 1000G/1000G.bed \
    --fam_file 1000G/1000G.QC.fam \
    --snp_file 1000G/common.snplist \
    --sub_bedfile 1000G/1000G.SUB.bed

INFO: Running subsets: 
PLINK v1.90b6.22 64-bit (16 Apr 2021)          www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 1000G/1000G.SUB.log.
Options in effect:
  --bfile 1000G/1000G
  --extract 1000G/common.snplist
  --keep 1000G/1000G.QC.fam
  --make-bed
  --out 1000G/1000G.SUB

8192 MB RAM detected; reserving 4096 MB for main workspace.
1664852 variants loaded from .bim file.
2490 people (1227 males, 1263 females) loaded from .fam.
--extract: 31566 variants remaining.
--keep: 2490 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2489 founders and 1 nonfounder present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
31566 variants and 2490 people pass filters and QC.
Note: No phenotypes pres

: 1

Totally 31566 varients

    ./plink \
        --bfile 1000G/1000G \
        --keep 1000G/1000G.QC.fam \
        --extract 1000G/common.snplist \
        --make-bed \
        --out 1000G/1000G.SUB


## Step 3: load data

In [6]:
sos run ldpred.ipynb data_load \
    --testpath "ukbiobank" \
    --ref_bfile 1000G/1000G.SUB.bed \
    --summstats_file mvpdata/pos_sumstats_hdl.SUB.rds \
    --ref_file 1000G/1000G.SUB.rds \
    --test_snplist UKB.SUB.snplist \
    --n_eff 200000

INFO: Running data_load_10: 04l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
Error : File '1000G/1000G.SUB.bk' already exists.
INFO: data_load_10 is completed.
INFO: Running data_load_20: 
INFO: data_load_20 (index=0) is ignored due to saved signature
INFO: data_load_20 output:   res-data/SumStats.RData
INFO: Running data_load_30: 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
31,566 variants to be matched.
1,134 ambiguous SNPs have been removed.
30,432 variants have been matched; 0 were flipped and 19,529 were reversed.
INFO: data_load_30 is completed.
INFO: data_load_30 output:   res-data/MatchedSnp.RData ukbioban

: 1

## Step 4: Quality control or do not

In [7]:
sos run ldpred.ipynb QControl \
    --qc_in res-data/MatchedSnp.RData

INFO: Running QControl: 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
   user  system elapsed 
  0.057   0.010   2.165 
Saving 7 x 7 in image
[1] "30376 over 30432 were removed in Quality Control."
INFO: QControl is c

: 1

## Step 5: subsetting test data

In [26]:
sos run ldpred.ipynb subsets \
    --bed_file ukbiobank/UKB.QC.bed \
    --fam_file ukbiobank/UKB.QC.fam \
    --snp_file ukbiobank/UKB.SUB.snplist \
    --sub_bedfile ukbiobank/UKB.SUB.bed

INFO: Running subsets: 
PLINK v1.90b6.22 64-bit (16 Apr 2021)          www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ukbiobank/UKB.SUB.log.
Options in effect:
  --bfile ukbiobank/UKB.QC
  --extract ukbiobank/UKB.SUB.snplist
  --keep ukbiobank/UKB.QC.fam
  --make-bed
  --out ukbiobank/UKB.SUB

8192 MB RAM detected; reserving 4096 MB for main workspace.
47605 variants loaded from .bim file.
2000 people (1085 males, 915 females) loaded from .fam.
--extract: 30432 variants remaining.
--keep: 2000 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2000 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
30432 variants and 2000 people pass filters and QC.
Note: No p

: 1

In [25]:
cd ukbiobank
./plink \
    --bfile UKB.QC \
    --extract UKB.SUB.snplist \
    --make-bed \
    --out UKB.SUB
cd ..

PLINK v1.90b6.22 64-bit (16 Apr 2021)          www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to UKB.SUB.log.
Options in effect:
  --bfile UKB.QC
  --extract UKB.SUB.snplist
  --make-bed
  --out UKB.SUB

8192 MB RAM detected; reserving 4096 MB for main workspace.
47605 variants loaded from .bim file.
2000 people (1085 males, 915 females) loaded from .fam.
--extract: 30432 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2000 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
30432 variants and 2000 people pass filters and QC.
Note: No phenotypes present.
--make-bed to UKB.SUB.bed + UKB.SUB.bim + UKB.SUB.fam ... 101112131415161718192021222324252627282930313233

: 1

## Step 6: Calculate LD matrix and correlation

In [9]:
sos run ldpred.ipynb LD \
    --ld_in res-data/MatchedSnp.Rdata

INFO: Running LD: 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Warning message:
In file.remove(paste0(tmp, ".sbk")) :
  cannot remove file './res-data/file1658859d4b4ad.sbk', reason 'No such file or directory'
INFO: LD is completed.
INFO: LD output:   res-data/LdMatrix.Rdata
INFO: Workflow LD (ID=wa8dcd8f85f92bd67) is executed successfully with 1 completed step.


: 1

## Step 7: Estimate posterior effect sizes and PRS

In [ ]:
sos run ldpred.ipynb load_testdata \
    --test_bfile ukbiobank/UKB.SUB.bed

In [64]:
sos run ldpred.ipynb inf \
    --inf_in res-data/LdMatrix.Rdata \
    --outpath "res-data" \
    --test_file ukbiobank/UKB.SUB.rds \
    --covariate ukbiobank/UKB.cov \
    --traits ukbiobank/UKB.hdl.cov

INFO: Running inf_10: l
INFO: inf_10 (index=0) is ignored due to saved signature
INFO: inf_10 output:   res-data/InfPred.Rdata
INFO: Running inf_20: 
INFO: inf_20 (index=0) is ignored due to saved signature
INFO: inf_20 output:   res-data/inf.cov
INFO: Workflow inf (ID=w1831d8e8c96f96ef) is ignored with 2 ignored steps.


: 1

In [2]:
sos run ldpred.ipynb grid:10 \
    --grid_in res-data/LdMatrix.Rdata \
    --outpath "res-data" \
    --test_file ukbiobank/UKB.SUB.rds

INFO: Running grid_10: 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
30,432 variants to be matched.
0 ambiguous SNPs have been removed.
30,432 variants have been matched; 0 were flipped and 29,999 were reversed.
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
INFO: grid_10 is completed.
INFO: grid_10 output:   res-data/GridPred.Rdata
INFO: Workflow grid (ID=w2b38d2f57b520df1) is executed successfully with 1 completed step.


: 1

In [3]:
sos run ldpred.ipynb auto:10 \
    --auto_in res-data/LdMatrix.Rdata \
    --outpath "res-data" \
    --test_file ukbiobank/UKB.SUB.rds

INFO: Running auto_10: 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
30,432 variants to be matched.
0 ambiguous SNPs have been removed.
30,432 variants have been matched; 0 were flipped and 29,999 were reversed.
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Savin

: 1

## Step 8: predict phenotypes

In [22]:
sos run ldpred.ipynb null_phenopred \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov

INFO: Running null_phenopred: 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
There were 50 or more warnings (use warnings() to see the first 50)
# A tibble: 1 x 3
  model          R2   MSE
  <chr>       <dbl> <dbl>
1 NULL model 0.0397 0.943
null device 
          1 
INFO: null_phenopred is completed.
INFO: null_phenopred output:   res-data/summary/NullSummary.pdf res-data/model/NullModel.Rdata
INFO: Workflow null_phenopred (ID=w267ecbfeb07e9d76) is executed successfully with 1 completed step.


: 1

In [30]:
sos run ldpred.ipynb inf_phenopred \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov \
    --inf_file res-data/InfPred.Rdata \
    --response 1

INFO: Running inf_phenopred: 4l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
There were 50 or more warnings (use warnings() to see the first 50)
# A tibble: 1 x 3
  model         R2   MSE
  <chr>      <dbl> <dbl>
1 Inf model 0.0399 0.944
null device 
          1 
INFO: inf_phenopred is completed.
INFO: inf_phenopred output:   res-data/summary/InfSummary.pdf res-data/model/InfModel.Rdata
INFO: Workflow inf_phenopred (ID=w155a831f53ecc4ca) is executed successfully with 1 completed step.


: 1

In [32]:
sos run ldpred.ipynb grid_phenopred \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov \
    --grid_file res-data/GridPred.Rdata \
    --response 1

INFO: Running grid_phenopred: l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
Saving 7 x 7 in image
There were 50 or more warnings (use warnings() to see the first 50)
# A tibble: 1 x 3
  model          R2   MSE
  <chr>       <dbl> <dbl>
1 Grid model 0.0405 0.948
pdf 
  2 
INFO: grid_phenopred is completed.
INFO: grid_phenopred output:   res-data/summary/GridSummary.pdf res-data/model/GridModel.Rdata... (3 items)
INFO: Workflow grid_phenopred (ID=w23f42d8d83262af5) is executed successfully with 1 completed step.


: 1

In [34]:
sos run ldpred.ipynb auto_phenopred \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov \
    --auto_file res-data/AutoPred.Rdata \
    --response 1

INFO: Running auto_phenopred: l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
There were 50 or more warnings (use warnings() to see the first 50)
# A tibble: 1 x 3
  model          R2   MSE
  <chr>       <dbl> <dbl>
1 Auto model 0.0398 0.944
null device 
          1 
INFO: auto_phenopred is completed.
INFO: auto_phenopred output:   res-data/summary/AutoSummary.pdf res-data/model/AutoModel.Rdata
INFO: Workflow auto_phenopred (ID=w48dc9ff9b8b99f04) is executed successfully with 1 completed step.


: 1